In [1]:
import os
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
FOLDER = '/content/gdrive/My Drive/Colab Notebooks/00.mycampus/[DL-202]Data/day1'
os.chdir(FOLDER)
print(os.getcwd())

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/00.mycampus/[DL-202]Data/day1


In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import tensorflow as tf

Using TensorFlow backend.


### LSTM 학습용을 위한 데이터 생성. window개념 포함.


In [2]:
## LSTM 학습용을 위한 데이터 생성. window개념 포함.
def getSeriesData(data, window_size, elementdim=1):
  '''
  window_size 만큼의 데이터로 다음 데이터를 예측하기 위한 학습데이터 생성
  :param data: [1,2,3,4,5,6]
  :param window_size: 3
  :param elementdim: 1
  :return: x=[[1,2,3], [2,3,4], [3,4,5]]
  :return: y=[      4,       5,       6]
  '''
  n_pairs = len(data) - window_size # (x,y) pair 개수
  xlist = [data[i:i+window_size] for i in range(n_pairs)]
  ylist = [data[i+window_size] for i in range(n_pairs)]
  x = np.array(xlist, dtype=np.float32)
  y = np.array(ylist, dtype=np.float32)
  return x, y

In [3]:
x, y = getSeriesData([1,2,3,4,5,6], 3)
print(x)
print(y)

[[1. 2. 3.]
 [2. 3. 4.]
 [3. 4. 5.]]
[4. 5. 6.]


In [4]:
# 학습용, 검증용, 테스트용 분할하기
def split_data(data, val_size=0.15, test_size=0.15):
  """
  splits data to training, validation and testing parts
  """
  ntest = int(round(len(data) * (1 - test_size)))
  nval = int(round(ntest * (1 - val_size)))
  
  train, validation, test = np.split(data, [nval, ntest])
  
  return train, validation, test

### 주가 데이터 csv파일 읽기

In [5]:
df = pd.read_csv('apple.csv', header=0)

In [6]:
#print(df)
df[:20] # jupyter에서는 print()없이 df라고만 쳐도 이쁘게 출력된다.

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-04,30.49,30.64,30.34,30.57,27.41,123432400.0
1,2010-01-05,30.66,30.80,30.46,30.63,27.45,150476200.0
2,2010-01-06,30.63,30.75,30.11,30.14,27.02,138040000.0
3,2010-01-07,30.25,30.29,29.86,30.08,26.97,119282800.0
4,2010-01-08,30.04,30.29,29.87,30.28,27.15,111902700.0
5,2010-01-11,30.40,30.43,29.78,30.02,26.91,115557400.0
6,2010-01-12,29.88,29.97,29.49,29.67,26.60,148614900.0
7,2010-01-13,29.70,30.13,29.16,30.09,26.98,151473000.0
8,2010-01-14,30.02,30.07,29.86,29.92,26.82,108223500.0
9,2010-01-15,30.13,30.23,29.41,29.42,26.37,148516900.0


### pandas를 이용한 데이터 처리: scaling

In [7]:
print('df.columns :\n', list(df.columns))
MAX_PRICE = df.High.max()
print('MAX_PRICE : ', MAX_PRICE)
df.High = df.High / MAX_PRICE


df.columns :
 ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
MAX_PRICE :  177.2


In [8]:
df[:10]

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-04,30.49,0.172912,30.34,30.57,27.41,123432400.0
1,2010-01-05,30.66,0.173815,30.46,30.63,27.45,150476200.0
2,2010-01-06,30.63,0.173533,30.11,30.14,27.02,138040000.0
3,2010-01-07,30.25,0.170937,29.86,30.08,26.97,119282800.0
4,2010-01-08,30.04,0.170937,29.87,30.28,27.15,111902700.0
5,2010-01-11,30.40,0.171727,29.78,30.02,26.91,115557400.0
6,2010-01-12,29.88,0.169131,29.49,29.67,26.60,148614900.0
7,2010-01-13,29.70,0.170034,29.16,30.09,26.98,151473000.0
8,2010-01-14,30.02,0.169695,29.86,29.92,26.82,108223500.0
9,2010-01-15,30.13,0.170598,29.41,29.42,26.37,148516900.0


### train, validation, test 데이터 준비
### LSTM 용으로 series 데이터 준비

In [9]:
df2 = df[df.Open != 0][['High']]
xy = df2.as_matrix() # numpy 데이터
print('xy.shape', xy.shape)

xy.shape (2021, 1)


C:\Miniconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [10]:
train, validation, test = split_data(xy)
WINDOWSIZE = 30
train_x, train_y = getSeriesData(train, WINDOWSIZE, elementdim=1)
valid_x, valid_y = getSeriesData(validation, WINDOWSIZE, elementdim=1)
test_x, test_y = getSeriesData(test, WINDOWSIZE, elementdim=1)

print('TRAIN', train.shape)
print('VALID', validation.shape)
print('TEST', test.shape)
print('TRAIN X', train_x.shape)
print('TRAIN Y', train_y.shape)
print('TEST X', test_x.shape)
print('TEST Y', test_y.shape)

TRAIN (1460, 1)
VALID (258, 1)
TEST (303, 1)
TRAIN X (1430, 30, 1)
TRAIN Y (1430, 1)
TEST X (273, 30, 1)
TEST Y (273, 1)


### LSTM NN 구성 정의

In [ ]:
# 모델 구성하기
model = Sequential()
hidden_nodes = 60
model.add(LSTM(hidden_nodes, input_shape=(WINDOWSIZE, 1)))
model.add(Dropout(0.3))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['MSE'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 60)                14880     
_________________________________________________________________
dropout_1 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 61        
Total params: 14,941
Trainable params: 14,941
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for layer in model.layers:
  print('IN', layer.input_shape, 'OUT', layer.output_shape)

IN (None, 30, 1) OUT (None, 60)
IN (None, 60) OUT (None, 60)
IN (None, 60) OUT (None, 1)


### 모델 학습하기
hist = model.fit(train_x, train_y, 
                 epochs=30, 
                 shuffle=True,
                 validation_data = (valid_x, valid_y),
                 verbose=1)

In [ ]:
scores = model.evaluate(test_x, test_y, verbose=1)

273/273 [==============================] - 0s 155us/step


In [ ]:
print(scores)

[0.0007748172538279941, 0.0007748172538279941]


In [ ]:
test_y2 = model.predict(test_x)
#print(test_y[:10, 2])
#print(test_y2[:10])
for y, y2 in zip(test_y[:50], test_y2[:50]):
    y = y*MAX_PRICE
    y2 = y2*MAX_PRICE
    print('%.1f %.1f %.2f%%' % (y, y2, abs(y-y2)/y*100))

115.0 109.6 4.74%
115.9 110.0 5.10%
116.2 110.6 4.80%
116.7 111.3 4.62%
116.5 112.1 3.78%
117.4 112.8 3.90%
117.5 113.5 3.39%
117.4 114.2 2.76%
116.5 114.7 1.55%
116.5 115.1 1.25%
117.8 115.3 2.15%
118.0 115.5 2.16%
117.1 115.7 1.24%
117.2 115.7 1.25%
116.3 115.7 0.51%
116.5 115.6 0.77%
116.9 115.4 1.22%
118.2 115.3 2.45%
119.4 115.2 3.53%
119.4 115.3 3.38%
119.9 115.6 3.63%
119.3 115.9 2.84%
119.6 116.2 2.83%
120.2 116.6 3.06%
120.5 116.9 2.98%
120.1 117.3 2.35%
120.5 117.6 2.40%
120.8 117.8 2.47%
120.1 118.1 1.68%
122.1 118.2 3.16%
122.4 118.5 3.22%
122.3 118.8 2.89%
121.6 119.1 2.06%
121.4 119.4 1.67%
130.5 119.5 8.42%
129.4 120.4 6.95%
129.2 121.5 5.95%
130.5 122.7 5.98%
132.1 124.0 6.14%
132.2 125.3 5.20%
132.4 126.7 4.37%
132.9 127.9 3.82%
133.8 128.9 3.65%
135.1 129.9 3.85%
136.3 130.8 4.00%
135.9 131.7 3.07%
135.8 132.5 2.45%
136.8 133.1 2.66%
137.1 133.7 2.52%
137.5 134.1 2.44%


### scale된 상태의 수치로 그래프

In [ ]:
fig, ax = plt.subplots()
ax.plot(test_y, label='actual') # 고가
ax.plot(test_y2, label='nn')
plt.legend()
plt.show()

### 원래의 수치로 변환 후 그래프

In [ ]:
fig, ax = plt.subplots()
ax.plot(test_y * MAX_PRICE, label='actual') # 고가
ax.plot(test_y2 * MAX_PRICE, label='nn')
plt.legend()
plt.show()